Lets do few shot inference against our EHR FHIR PostGre SQL database



1.   Load and test Open AI access
2.   Load and test EHR PostgreSQL database access
3.   Load FAISS









# 0. Mount drive and define paths

# Installs

In [1]:
# --- 0) Mount Drive and set your variables ---
from google.colab import drive
drive.mount('/content/drive')

DEV_PATH = "/content/drive/MyDrive/210_Capstone/210_Factory/210_dev"
FAISS_DB_PATH = DEV_PATH + "/vectorstores/medintellagent_faiss_v1"
POSTGRES_DB_PATH = DEV_PATH + "/synthea_ehr_backup.sql"  # (dump file; not used in this snippet)
LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-large"



Mounted at /content/drive


In [2]:
%%capture
# --- 1) Install libraries (once per runtime) ---
!pip -q install --upgrade openai langchain langchain-community langchain-openai faiss-cpu


In [3]:
# --- 2) Load your OpenAI key (recommended: Colab "Secrets" → OPENAI_API_KEY) ---
import os
try:
    from google.colab import userdata
    key = userdata.get('OPENAI_API_KEY')
    if key: os.environ['OPENAI_API_KEY'] = key
except Exception:
    pass

if not os.environ.get("OPENAI_API_KEY"):
    import getpass
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY: ")

# Initialize OpenAI client
from openai import OpenAI
client = OpenAI()


In [4]:
# --- 3) Load  FAISS vector store (must use same embedding model used to build it) ---
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vs = FAISS.load_local(FAISS_DB_PATH, embeddings, allow_dangerous_deserialization=True)

print("FAISS loaded. Example count:", len(vs.docstore._dict))


FAISS loaded. Example count: 90


In [5]:
# --- Prompt building helpers ---

# Keep this aligned to your actual DB schema + rules
PREFIX = (
    "Return a single PostgreSQL SELECT only.\n"
    "Use only tables: patients, encounters, conditions, observations, medication_requests, procedures.\n"
    "Use only parameter :patient_id. Prefer DISTINCT ON with ORDER BY for 'latest per X'; no CTEs or window functions.\n"
    "Do not mix GROUP BY with DISTINCT ON. If aggregation is needed (e.g., pairing BP), use GROUP BY + MAX(CASE...).\n"
    "Important: medication_requests has no rxnorm_code (use med_name only). Encounters has no location.\n"
    "If the question mentions 'blood pressure' or 'BP', return only systolic (8480-6) and diastolic (8462-4) results and prefer paired rows grouped by effective_datetime.\n"
    "\n"
    "Schema hints:\n"
    "  conditions(display, code, onset_datetime, abatement_datetime, encounter_id, patient_id, condition_id)\n"
    "  observations(display, loinc_code, value_num, value_unit, effective_datetime, encounter_id, patient_id, observation_id)\n"
    "  medication_requests(med_name, dose, route, start_datetime, end_datetime, refills, encounter_id, patient_id, med_request_id)\n"
    "  encounters(start_datetime, end_datetime, reason_text, class, encounter_id, patient_id)\n"
    "  procedures(display, code, performed_datetime, encounter_id, patient_id, procedure_id)\n"
    "Output only the raw SQL, no markdown fences."
)

def get_few_shots(query: str, k: int = 3):
    # uses your already-loaded FAISS vector store: `vs`
    docs_scores = vs.similarity_search_with_score(query, k=k)
    examples = [{"question": d.page_content, "sql": (d.metadata or {}).get("sql","")} for d, _ in docs_scores]
    return examples

def format_examples(examples):
    return "\n".join([f"Question: {ex['question']}\nSQL:\n{ex['sql']}\n" for ex in examples])

def build_prompt(user_question: str, k: int = 3) -> str:
    examples = get_few_shots(user_question, k=k)
    return f"{PREFIX}\n{format_examples(examples)}\nQuestion: {user_question}\nSQL:"


In [6]:
# --- 5) SQL generation + a tiny safety check ---
import re

SELECT_ONLY = re.compile(r"^\s*select\b", re.IGNORECASE | re.DOTALL)

def clean_sql(text: str) -> str:
    s = text.strip()

    # strip a leading "SQL:" line if present
    if s.lower().startswith("sql:"):
        s = s[4:].strip()

    # strip fenced code blocks like ```sql ... ``` or ``` ... ```
    m = re.match(r"^```(?:\s*sql)?\s*([\s\S]*?)\s*```$", s, flags=re.IGNORECASE)
    if m:
        s = m.group(1).strip()

    # strip stray backticks if the model emitted them oddly
    if s.startswith("```") and "```" in s[3:]:
        s = s.split("```", 1)[1].rsplit("```", 1)[0].strip()

    # remove BOM or weird invisibles
    s = s.replace("\ufeff", "").replace("\u200b", "").strip()
    return s

def is_safe_select(text: str) -> bool:
    sql = clean_sql(text)

    # (optional) reject multiple statements (allow a single trailing semicolon)
    trimmed = sql.strip()
    if ";" in trimmed[:-1]:  # semicolon before the last char
        return False

    if not SELECT_ONLY.match(trimmed):
        return False

    banned = (" insert ", " update ", " delete ", " drop ", " alter ",
              " create ", " grant ", " revoke ", " truncate ")
    low = f" {trimmed.lower()} "  # pad with spaces to avoid substring accidents
    return not any(b in low for b in banned)

def generate_sql(user_question: str, k: int = 3, max_tokens: int = 400):
    prompt = build_prompt(user_question, k=k)
    resp = client.chat.completions.create(
        model=LLM_MODEL,
        temperature=0,
        messages=[
            {"role":"system","content":"You are a precise SQL generator for a patient portal."},
            {"role":"user","content": prompt}
        ],
        max_tokens=max_tokens,
    )
    sql = resp.choices[0].message.content.strip()
    return sql

# Demo
demo_q = "Which medications am I currently taking?"
sql = generate_sql(demo_q, k=3)
print(sql, "\n\nSAFE:", is_safe_select(sql))


SELECT DISTINCT ON (mr.patient_id, mr.med_name)
  mr.patient_id,
  mr.med_name AS medication,
  mr.dose,
  mr.route,
  mr.start_datetime,
  mr.end_datetime,
  mr.refills
FROM medication_requests mr
WHERE mr.patient_id = :patient_id
  AND (mr.end_datetime IS NULL OR mr.end_datetime >= NOW())
ORDER BY mr.patient_id,
         mr.med_name,
         COALESCE(mr.end_datetime, mr.start_datetime) DESC NULLS LAST; 

SAFE: True


# Load PostgreSQL EHR FHIR Database

In [7]:
%%capture
!apt-get -y update
!apt-get -y install postgresql postgresql-contrib

!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres createdb synthea_ehr

!echo "PostgreSQL installed, service started, user password set to 'postgres', and DB 'synthea_ehr' created."


In [8]:
import subprocess
import os

# Database connection details
DB_NAME = "synthea_ehr"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"

# Path on Google Drive to the backup file
BACKUP_PATH = DEV_PATH + "/synthea_ehr_backup.sql"

def restore_database():
    """Restores the synthea_ehr database from a file on Google Drive."""
    try:
        print("Starting database restore...")

        # First, drop and re-create the database to ensure a clean state
        print("Dropping and re-creating the database for a clean restore...")
        env = os.environ.copy()
        env['PGPASSWORD'] = DB_PASSWORD

        # Command to drop the database
        drop_command = [
            'dropdb',
            '--host', DB_HOST,
            '--username', DB_USER,
            DB_NAME
        ]
        # This will fail if the DB doesn't exist, so we don't check for errors
        subprocess.run(drop_command, env=env, check=False, capture_output=True, text=True)

        # Command to create the database
        create_command = [
            'createdb',
            '--host', DB_HOST,
            '--username', DB_USER,
            DB_NAME
        ]
        subprocess.run(create_command, env=env, check=True, capture_output=True, text=True)
        print("Database re-created successfully.")

        # Use subprocess to run the psql command to restore the backup
        command = [
            'psql',
            '--host', DB_HOST,
            '--username', DB_USER,
            '--dbname', DB_NAME,
            '--file', BACKUP_PATH
        ]

        process = subprocess.run(command, env=env, check=True, capture_output=True, text=True)
        print("Database restore successful!")

    except FileNotFoundError:
        print("Error: psql or dropdb/createdb commands not found. Please ensure PostgreSQL client tools are installed.")
        print("You can try running: !apt-get update && !apt-get install -y postgresql-client")
    except subprocess.CalledProcessError as e:
        print("Error during restore process.")
        print(f"Subprocess returned non-zero exit code: {e.returncode}")
        print(f"STDOUT:\n{e.stdout}")
        print(f"STDERR:\n{e.stderr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

restore_database()

Starting database restore...
Dropping and re-creating the database for a clean restore...
Database re-created successfully.
Database restore successful!


In [9]:
# If needed once per runtime:
# !pip -q install psycopg2-binary

import re
import psycopg2
import psycopg2.extras

# Uses your existing globals:
# DB_NAME = "synthea_ehr"
# DB_USER = "postgres"
# DB_PASSWORD = "postgres"
# DB_HOST = "localhost"

# --- helpers ---
_SELECT_ONLY = re.compile(r"^\s*select\b", re.IGNORECASE | re.DOTALL)
_BANNED = (" insert ", " update ", " delete ", " drop ", " alter ",
           " create ", " grant ", " revoke ", " truncate ", " copy ", " do ")

# match :name (not preceded by another :)
_PARAM = re.compile(r'(?<!:):([a-zA-Z_]\w*)')

def _clean_sql(text: str) -> str:
    """Remove code fences / labels and invisible chars."""
    s = (text or "").strip()
    if s.lower().startswith("sql:"):
        s = s[4:].strip()
    m = re.match(r"^```(?:\s*sql)?\s*([\s\S]*?)\s*```$", s, flags=re.IGNORECASE)
    if m:
        s = m.group(1).strip()
    return s.replace("\ufeff","").replace("\u200b","").strip()

def _is_safe_select(sql: str) -> bool:
    """Single SELECT only; no DDL/DML keywords; no mid-string semicolons."""
    s = sql.strip()
    if ";" in s[:-1]:  # allow a single trailing semicolon only
        return False
    if not _SELECT_ONLY.match(s):
        return False
    low = f" {s.lower()} "
    return not any(b in low for b in _BANNED)

def _to_psycopg2_named(sql: str) -> str:
    """Convert :name placeholders to %(name)s for psycopg2."""
    return _PARAM.sub(r"%(\1)s", sql)

# --- main function ---
def execute_sql(sql: str, params: dict = None, timeout_ms: int = 5000):
    """
    Execute a single SELECT query safely and return rows as a list of dicts.

    Args:
      sql: SQL string (can use :param style placeholders, e.g., :patient_id)
      params: dict of parameters if placeholders are used (optional)
      timeout_ms: statement timeout in milliseconds (default 5000)

    Returns:
      List[Dict]: each row as a dict
    """
    raw = _clean_sql(sql)
    if not _is_safe_select(raw):
        raise ValueError("Blocked: SQL must be a single SELECT without DDL/DML.")

    q = _to_psycopg2_named(raw)
    params = params or {}

    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=5432,
        connect_timeout=5,
        options=f"-c statement_timeout={timeout_ms}"
    )
    try:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(q, params)
            return [dict(r) for r in cur.fetchall()]
    finally:
        conn.close()


In [10]:
# Test

# Simple
rows = execute_sql("SELECT COUNT(*) AS n FROM patients;")
print(rows)


[{'n': 111}]


In [11]:
rows = execute_sql("select patient_id from patients;")
print(rows[:3])

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'}, {'patient_id': '782001bc-f712-50ae-04f5-9a488f3ef4aa'}, {'patient_id': '80e7f50a-3e99-d5ac-cf97-f8a4b4f9e6c7'}]


In [12]:
execute_sql("""
SELECT column_name
FROM information_schema.columns
WHERE table_schema='public' AND table_name='encounters'
ORDER BY 1;
""")

[{'column_name': 'class'},
 {'column_name': 'encounter_id'},
 {'column_name': 'end_datetime'},
 {'column_name': 'patient_id'},
 {'column_name': 'reason_text'},
 {'column_name': 'start_datetime'}]

In [13]:
execute_sql("""
SELECT table_name, column_name
FROM information_schema.columns
WHERE table_schema='public'
ORDER BY 1;
""")

[{'table_name': 'conditions', 'column_name': 'onset_datetime'},
 {'table_name': 'conditions', 'column_name': 'patient_id'},
 {'table_name': 'conditions', 'column_name': 'encounter_id'},
 {'table_name': 'conditions', 'column_name': 'code'},
 {'table_name': 'conditions', 'column_name': 'display'},
 {'table_name': 'conditions', 'column_name': 'condition_id'},
 {'table_name': 'conditions', 'column_name': 'abatement_datetime'},
 {'table_name': 'encounters', 'column_name': 'reason_text'},
 {'table_name': 'encounters', 'column_name': 'start_datetime'},
 {'table_name': 'encounters', 'column_name': 'end_datetime'},
 {'table_name': 'encounters', 'column_name': 'encounter_id'},
 {'table_name': 'encounters', 'column_name': 'patient_id'},
 {'table_name': 'encounters', 'column_name': 'class'},
 {'table_name': 'immunizations', 'column_name': 'base_cost'},
 {'table_name': 'immunizations', 'column_name': 'code'},
 {'table_name': 'immunizations', 'column_name': 'date'},
 {'table_name': 'immunizations', 

In [14]:
# With parameter (your canonical pattern)
q = """
SELECT DISTINCT ON (mr.patient_id, mr.med_name)
  mr.patient_id,
  mr.med_name AS medication,
  mr.dose,
  mr.route,
  mr.start_datetime,
  mr.end_datetime,
  mr.refills
FROM medication_requests mr
WHERE mr.patient_id = :patient_id
  AND (mr.end_datetime IS NULL OR mr.end_datetime >= NOW())
ORDER BY mr.patient_id,
         mr.med_name,
         COALESCE(mr.end_datetime, mr.start_datetime) DESC NULLS LAST;

"""
rows = execute_sql(q, {"patient_id": '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'})
print(rows[:3])

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40', 'medication': 'Acetaminophen 325 MG Oral Tablet', 'dose': None, 'route': None, 'start_datetime': None, 'end_datetime': None, 'refills': None}, {'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40', 'medication': 'Naproxen sodium 220 MG Oral Tablet', 'dose': None, 'route': None, 'start_datetime': None, 'end_datetime': None, 'refills': None}]


# Tie LLM output to return results from PostGre SQL database

In [15]:
def answer_patient_question(user_question: str, patient_id: str, k: int = 3, max_tokens: int = 400 ):
    sql = generate_sql(user_question, k=k, max_tokens=max_tokens)
    #print("answer_patient_question: Generated SQL:\n", sql, "\n")
    rows = execute_sql(sql, {"patient_id": patient_id})
    #print("answer_patient_question: Executed SQL returned rows:\n", len(rows), "\n")
    return sql, rows


In [16]:
execute_sql("select p.patient_id from patients p")

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'},
 {'patient_id': '782001bc-f712-50ae-04f5-9a488f3ef4aa'},
 {'patient_id': '80e7f50a-3e99-d5ac-cf97-f8a4b4f9e6c7'},
 {'patient_id': 'edc17058-55fb-08c7-12df-ece93a402e50'},
 {'patient_id': '9f9dbdcb-23a1-82cc-b7bc-e0e420a95bd1'},
 {'patient_id': 'be874504-c868-ebfd-9a77-df6b1e5ff6cc'},
 {'patient_id': '30e48e16-2df7-207e-7a3d-1650ef0c1ed8'},
 {'patient_id': '57b21dea-ff00-6c3e-92d9-91c7627f53b2'},
 {'patient_id': 'a3d34c1f-5421-e078-38ec-1498a5941dbe'},
 {'patient_id': 'e83fe1b3-f94f-5591-f851-1da300e24e99'},
 {'patient_id': 'e6705c33-7349-8b12-484d-3b1f93227178'},
 {'patient_id': '2da86d63-34ae-b887-ddff-8f6f1e6990f1'},
 {'patient_id': '04181caa-fcc1-c6c8-743e-a903eff368de'},
 {'patient_id': '20802592-1c31-7339-4c4c-2fe648e1a716'},
 {'patient_id': '406e8bad-81b5-7624-5b8a-4aeeb74028f5'},
 {'patient_id': 'a331b5bc-cbea-a205-a8bf-dbf3255ef36a'},
 {'patient_id': '641efcda-7397-4172-c6ac-8231342fa53e'},
 {'patient_id': 'e64918a6-528c-

In [17]:
sql = (
    "SELECT e.patient_id, e.start_datetime, e.end_datetime, "
    "e.class AS encounter_class, e.reason_text AS reason "
    "FROM encounters e "
    "WHERE e.reason_text IS NOT NULL;"
)
execute_sql(sql)

[]

In [18]:
import pandas as pd

patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = "List all my vital signs"

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
print("Generated SQL:\n", sql, "\n")
print("Rows:", len(rows))
if rows:
    display(pd.DataFrame(rows).head(10))

Generated SQL:
 SELECT
  o.patient_id,
  COALESCE(o.display, o.loinc_code) AS vital_name,
  o.value_num AS value,
  o.value_unit AS unit,
  o.effective_datetime
FROM observations AS o
WHERE o.patient_id = :patient_id
  AND (
    o.loinc_code IN ('8480-6','8462-4','8867-4','9279-1','8310-5','59408-5','29463-7','39156-5','8302-2')
    OR LOWER(o.display) IN (
      'systolic blood pressure','diastolic blood pressure',
      'heart rate','respiratory rate','body temperature',
      'oxygen saturation','body weight','bmi','body mass index','body height'
    )
  )
ORDER BY
  o.effective_datetime DESC NULLS LAST,
  COALESCE(o.display, o.loinc_code); 

Rows: 56


,patient_id,vital_name,value,unit,effective_datetime
0,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Height,183.4,cm,2025-04-21 14:19:34+00:00
1,0fca905f-391c-08d3-4b93-b53f69b9da53,Body mass index (BMI) [Ratio],28.35,kg/m2,2025-04-21 14:19:34+00:00
2,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Weight,95.4,kg,2025-04-21 14:19:34+00:00
3,0fca905f-391c-08d3-4b93-b53f69b9da53,Heart rate,69,/min,2025-04-21 14:19:34+00:00
4,0fca905f-391c-08d3-4b93-b53f69b9da53,Respiratory rate,14,/min,2025-04-21 14:19:34+00:00
5,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Height,183.4,cm,2024-04-15 14:19:34+00:00
6,0fca905f-391c-08d3-4b93-b53f69b9da53,Body mass index (BMI) [Ratio],28.26,kg/m2,2024-04-15 14:19:34+00:00
7,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Weight,95.1,kg,2024-04-15 14:19:34+00:00
8,0fca905f-391c-08d3-4b93-b53f69b9da53,Heart rate,83,/min,2024-04-15 14:19:34+00:00
9,0fca905f-391c-08d3-4b93-b53f69b9da53,Respiratory rate,15,/min,2024-04-15 14:19:34+00:00


# Post data frame, LLM genberates a nice summary

In [19]:
import pandas as pd
from openai import OpenAI
import io

client = OpenAI()  # assumes OPENAI_API_KEY is set


In [20]:
def df_to_csv_for_llm(df: pd.DataFrame, max_rows: int = 200, null_marker: str = "—") -> tuple[str, bool]:
    """
    Convert a DataFrame to CSV for the LLM.
    - Truncates to max_rows to keep prompts small.
    - Replaces NaNs with a visible marker (default "—").
    Returns (csv_text, truncated_flag).
    """
    truncated = False
    if len(df) > max_rows:
        df = df.head(max_rows).copy()
        truncated = True

    df = df.copy()
    df = df.fillna(null_marker)

    # keep column order stable
    csv_buf = io.StringIO()
    df.to_csv(csv_buf, index=False)
    return csv_buf.getvalue(), truncated

def summarize_df_with_llm(
    df: pd.DataFrame,
    patient_id: str,
    model: str = "gpt-4o-mini",
    max_rows: int = 200,
    null_marker: str = "—",
    max_tokens: int = 500
) -> str:
    """
    Ask the LLM to summarize a DataFrame.
    - Includes all visible (non-missing) values in its reasoning context,
      but the model will produce a concise natural-language summary (not a reprint of all rows).
    """
    if df is None or df.empty:
        return "No data found for the requested query."

    csv_text, truncated = df_to_csv_for_llm(df, max_rows=max_rows, null_marker=null_marker)
    columns_csv = ",".join(list(df.columns))

    user_prompt = f"""
You are a precise medical data summarizer. Use only the table below.
- Do not invent values or fields.
- Call out trends, counts, notable recency, and any obvious gaps (fields marked "{null_marker}").
- Keep it concise (2–5 sentences).
- If the table was truncated, say so and include how many rows were shown.

Patient: {patient_id}
Columns: {columns_csv}
Rows shown: {min(len(df), max_rows)}{' (truncated)' if truncated else ''}

CSV:
{csv_text}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a precise, conservative medical data summarizer."},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=max_tokens,
    )
    return resp.choices[0].message.content.strip()


In [21]:
# ##
# patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
# user_q = "What has been my highest weight"

# sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
# print("Generated SQL:\n", sql, "\n")
# print("Rows:", len(rows))
# # if rows:
# #     display(pd.DataFrame(rows).head(10))
# df = pd.DataFrame(rows)
# display(df.head(10))
# # 3) Summarize with the LLM
# summary = summarize_df_with_llm(df, patient_id="<REAL-PATIENT-ID>", model=LLM_MODEL)
# print(summary)

# Gradio UI

In [22]:
# 0) Install Gradio (quiet)
!pip -q install gradio

# 1) Build a small adapter for the UI
import pandas as pd
import gradio as gr

def medintellagent_ui(patient_id: str, user_question: str, k: int = 3, max_tokens: int = 600):
    if not patient_id or not user_question:
        return "Please provide both patient_id and a question.", pd.DataFrame(), "—"
    try:
        # Generate SQL + fetch rows
        sql, rows = answer_patient_question(user_question, patient_id, k=k, max_tokens=max_tokens)
        df = pd.DataFrame(rows)
        if df.empty:
            summary = "No data found for this query (for this patient). Try another question or patient_id."
        else:
            summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
        return sql, df, summary
    except Exception as e:
        # Show errors cleanly in the UI (helpful during dev)
        return f"Error: {e}", pd.DataFrame(), "—"

# 2) Build the interface
with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown("# MedIntellAgent — MVP UI")
    gr.Markdown("Enter a **Patient ID** and **Question**. You’ll get the generated SQL, raw DB rows, and an LLM summary.")

    with gr.Row():
        patient = gr.Textbox(label="Patient ID", placeholder="e.g., 0fca905f-391c-08d3-4b93-b53f69b9da53")
    question = gr.Textbox(label="Patient Question", lines=2,
                          placeholder="e.g., Which medications am I currently taking?")
    with gr.Row():
        k = gr.Slider(1, 5, value=3, step=1, label="Few-shot k")
        max_tokens = gr.Slider(100, 2000, value=600, step=50, label="LLM max_tokens")

    ask = gr.Button("Ask MedIntellAgent")

    sql_out = gr.Code(label="Generated SQL", language="sql")
    table_out = gr.Dataframe(label="Postgres Results", interactive=False)
    summary_out = gr.Markdown(label="LLM Summary")

    ask.click(medintellagent_ui,
              inputs=[patient, question, k, max_tokens],
              outputs=[sql_out, table_out, summary_out])

demo.launch(share=True)   # share=True if you want a temporary public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://65f2b69302b2b7e65d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Evaluation



1.   For now we will use Open AI as LLM for generating Optimized response summary, and score evaluation. Down the line we may change this to e.g. Calude




In [23]:
%%capture
pip install -U langgraph

In [30]:
%%capture
pip install -U langchain_core langchain_openai

In [31]:
from pydantic import BaseModel, Field

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [50]:
from typing import TypedDict
from typing import Literal
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate


def generate_empathy_score(user_query: str, response: str):

    #Graph State
    class State(TypedDict):
      user_query: str
      response: str
      empathy_score: str
      is_empathetic: bool


    #Schema for structured outout to use in evaluation
    class Feedback(BaseModel):
      score: Literal ["1","2","3","4","5"] = Field(
          description="Provide a score for how empathetic is the response. 5 is highest empathy."
      )
      feedback: str = Field(
          description="Provide a score for how empathetic is the response. Score of 1: response has no empathy. Score of 5: response has excellent empathy for a human."

      )

    evaluator = llm.with_structured_output(Feedback)

    seeded_data = {
    "user_query": user_query,
    "response": response
    }
    # --- 3. CREATE PROMPT TEMPLATE ---

    # Define the evaluation instructions using a system message and placeholders
    prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are an expert, highly empathetic evaluator.
                Your task is to judge the system generated response to a user (patient) query.
                You must provide a score (1-5) and detailed feedback on the empathy level of the response, strictly following the provided JSON schema.

                --- Context ---
                User Query: {user_query}
                Response to Evaluate: {response}
                """
            ),
            ("human", "Evaluate the response for empathy and provide the JSON output.")
        ]
    )

    # Prepare the data for the prompt
    # We convert the table to a string for the prompt
    prompt_data = {
        "user_query": seeded_data["user_query"],
        "response": seeded_data["response"]
    }

    # --- 4. RUN EVALUATION ---

    # Create the runnable chain: Prompt -> LLM with Structured Output
    evaluation_chain = prompt_template | evaluator

    # Invoke the chain
    try:
        evaluation_result: Feedback = evaluation_chain.invoke(prompt_data)

        # Output the result
        print("--- EVALUATION RESULT ---")
        print(f"Empathy Score: {evaluation_result.score}/5")
        print(f"Feedback: {evaluation_result.feedback}")

        return {"empathy_score": evaluation_result.score, "feedback": evaluation_result.feedback}
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")




In [51]:
#Testing of empathy score

patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'which medication am I taking?'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)


In [52]:
summary

'The table shows medication records for a single patient (ID: 0fca905f-391c-08d3-4b93-b53f69b9da53) with five entries. All medications listed, including Acetaminophen and amLODIPine, have a dose of 1.0, but the route, start and end dates, and refills are not provided (marked as "—"). Notably, there is a gap in the data regarding the administration details and refill information, which limits the understanding of the patient\'s medication regimen. The table is truncated, showing only 5 rows.'

In [53]:
generate_empathy_score(user_q,summary)

--- EVALUATION RESULT ---
Empathy Score: 1/5
Feedback: The response lacks any empathetic language or acknowledgment of the user's situation. It is purely factual and does not address the user's need for clarity or support regarding their medication. There is no attempt to connect with the user emotionally or to provide reassurance, which is crucial in a healthcare context.


{'empathy_score': '1',
 'feedback': "The response lacks any empathetic language or acknowledgment of the user's situation. It is purely factual and does not address the user's need for clarity or support regarding their medication. There is no attempt to connect with the user emotionally or to provide reassurance, which is crucial in a healthcare context."}

In [55]:
user_q = 'List all my vital signs'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [56]:
summary

'The data for patient 0fca905f-391c-08d3-4b93-b53f69b9da53 includes 56 rows of vital signs, with the most recent measurements taken on April 21, 2025. Notable trends include a decrease in Body Weight from 102 kg in April 2023 to 95.4 kg in April 2025, and a corresponding decrease in Body Mass Index (BMI) from 30.33 kg/m² to 28.35 kg/m². Heart rate has fluctuated, with a notable drop from 88 beats per minute in April 2023 to 69 in April 2025. There are gaps in the data for Body Temperature, which is only recorded once in June 2017. The table appears to be truncated, showing only 56 rows.'

In [57]:
generate_empathy_score(user_q,summary)

--- EVALUATION RESULT ---
Empathy Score: 2/5
Feedback: The response provides factual information about the user's vital signs but lacks empathy and personal connection. It presents data in a clinical manner without acknowledging the user's potential feelings or concerns regarding their health. There is no expression of understanding or support, which is crucial when discussing personal health information. A more empathetic response would include reassurance, an invitation for the user to ask questions, or acknowledgment of the significance of these changes in their health.


{'empathy_score': '2',
 'feedback': "The response provides factual information about the user's vital signs but lacks empathy and personal connection. It presents data in a clinical manner without acknowledging the user's potential feelings or concerns regarding their health. There is no expression of understanding or support, which is crucial when discussing personal health information. A more empathetic response would include reassurance, an invitation for the user to ask questions, or acknowledgment of the significance of these changes in their health."}

In [58]:
user_q = 'what has been my weight history'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [59]:
summary

'The data shows a trend of fluctuating body weight for the patient over several years, with the most recent measurement on April 21, 2025, at 95.4 kg. Notably, there was a peak weight of 102 kg recorded in both April 2023 and March 2018. The values indicate a general decrease in weight from 102 kg in 2023 to 95.4 kg in 2025, suggesting a potential weight loss trend. There are no gaps in the vital name or unit fields, but the effective_datetime shows a consistent annual measurement pattern, with the most recent data being from the future, indicating a possible data entry error. The table is truncated, showing 11 rows.'

In [60]:
score_response = generate_empathy_score(user_q,summary)

--- EVALUATION RESULT ---
Empathy Score: 2/5
Feedback: The response provides factual information about the user's weight history but lacks empathy and emotional connection. It presents data in a clinical manner without acknowledging the user's potential feelings or concerns regarding their weight fluctuations. There is no expression of understanding or support, which is crucial when discussing personal health matters. A more empathetic response would include validation of the user's experience and encouragement regarding their weight loss trend.


In [61]:
print(score_response)

{'empathy_score': '2', 'feedback': "The response provides factual information about the user's weight history but lacks empathy and emotional connection. It presents data in a clinical manner without acknowledging the user's potential feelings or concerns regarding their weight fluctuations. There is no expression of understanding or support, which is crucial when discussing personal health matters. A more empathetic response would include validation of the user's experience and encouragement regarding their weight loss trend."}


# Now after these trial runs we aspire to optimize the empathy score

In [92]:
from typing import TypedDict, Literal, Optional
import pandas as pd
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage # NEW IMPORT
from langgraph.graph import StateGraph, START, END

# --- 1. CONFIGURATION AND SCHEMA ---

# Note: This requires the OPENAI_API_KEY environment variable to be set.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1.1 Graph State (TypedDict)
class PatientState(TypedDict):
    """The state of the conversation and evaluation."""
    patient_query: str
    patient_data: str # Using str to hold Markdown representation of DataFrame
    response: str
    empathy_score: int
    evaluation_feedback: str
    retry_count: int

# 1.2 Structured Output Schema (Pydantic)
class EmpathyEvaluation(BaseModel):
    """Schema for the structured output of the evaluation LLM."""
    score: Literal[1, 2, 3, 4, 5] = Field(
        description="The empathy score for the response (1=low empathy, 5=high empathy)."
    )
    feedback: str = Field(
        description="Constructive, actionable feedback on how to improve the response's empathy, especially if the score is 3 or less."
    )

# Augment the LLM for structured evaluation output
evaluator_llm = llm.with_structured_output(EmpathyEvaluation)


# --- 2. GRAPH NODES ---

def generate_response(state: PatientState) -> dict:
    """
    Node 1: Generates or regenerates a patient response using the LLM.
    """
    print(f"\n--- GENERATING RESPONSE (Retry: {state['retry_count']}) ---")

    # Base instructions content
    system_prompt_content = (
        "You are a highly empathetic and professional clinical assistant. "
        "Your primary goal is to acknowledge the patient's feelings and provide clear, gentle, and helpful next steps. "
        "The current patient query and their data are provided below. Respond with empathy first."
    )

    # Context message content (always included)
    context_message_content = (
        f"Patient Query: {state['patient_query']}\nPatient Data Snapshot:\n{state['patient_data']}"
    )

    # Determine user prompt content
    if state["retry_count"] > 0:
        # Retry prompt: incorporate previous feedback
        user_prompt_content = (
            f"RETRY RESPONSE. The previous attempt scored {state['empathy_score']}/5. "
            f"The feedback was: '{state['evaluation_feedback']}'. "
            f"Write a new response that explicitly addresses this feedback and improves empathy."
        )
    else:
        # Initial prompt
        user_prompt_content = "Generate a response."

    # Construct the final list of BaseMessages (REQUIRED INPUT TYPE)
    messages = [
        SystemMessage(content=system_prompt_content),
        HumanMessage(content=context_message_content),
        HumanMessage(content=user_prompt_content)
    ]

    # Invoke LLM with the list of messages
    msg = llm.invoke(messages)

    return {
        "response": msg.content,
        "retry_count": state["retry_count"] + 1,
        "evaluation_feedback": "" # Clear feedback before next evaluation
    }


def evaluate_response(state: PatientState) -> dict:
    """
    Node 2: Evaluates the generated response for empathy using the structured LLM.
    """
    print("\n--- EVALUATING RESPONSE ---")

    # Prompt for the structured evaluator
    evaluation_prompt = (
        f"Evaluate the following clinical assistant response for empathy on a scale of 1 to 5. "
        f"Score 1 means cold/robotic. Score 5 means outstanding empathy and warmth. "
        f"Patient Query: {state['patient_query']}\n"
        f"Assistant Response: {state['response']}"
    )

    # Invoke the structured LLM
    evaluation_result: EmpathyEvaluation = evaluator_llm.invoke(evaluation_prompt)

    print(f"-> Score: {evaluation_result.score}/5")

    return {
        "empathy_score": evaluation_result.score,
        "evaluation_feedback": evaluation_result.feedback
    }


def route_to_retry(state: PatientState) -> str:
    """
    Conditional Edge: Decides whether to loop back for regeneration or end.
    """
    score = state["empathy_score"]
    max_retries = 3 # Safety limit to prevent infinite loops

    if score > 3:
        print("\n*** ROUTE: END (Score > 3) ***")
        return "end"
    elif state["retry_count"] >= max_retries:
        print(f"\n*** ROUTE: END (Max Retries ({max_retries}) Reached) ***")
        return "end"
    else:
        print("\n*** ROUTE: RETRY (Score <= 3) ***")
        return "retry"


# --- 3. WORKFLOW ASSEMBLY AND MAIN FUNCTION ---

def optimize_patient_response(query: str, data: pd.DataFrame, max_retries=3):
    """
    Main function to run the optimization graph.

    Args:
        query (str): The patient's input question or concern.
        data (pd.DataFrame): Relevant patient data.
        max_retries (int): Maximum number of attempts before stopping.

    Returns:
        dict: The final state of the graph, including the optimized response.
    """

    # Initialize the graph builder
    optimizer_builder = StateGraph(PatientState)

    # Add the nodes (actions)
    optimizer_builder.add_node("generator", generate_response)
    optimizer_builder.add_node("evaluator", evaluate_response)

    # Define the entry point
    optimizer_builder.add_edge(START, "generator")

    # Define the flow from generation to evaluation
    optimizer_builder.add_edge("generator", "evaluator")

    # Define the conditional loop after evaluation
    optimizer_builder.add_conditional_edges(
        "evaluator",
        route_to_retry,
        {
            "retry": "generator", # Loop back to generator
            "end": END,           # Stop the workflow
        }
    )

    # Compile the workflow
    optimizer_workflow = optimizer_builder.compile()

    # Initial state preparation
    initial_state = {
        "patient_query": query,
        "patient_data": data.to_markdown(), # Convert DataFrame head to readable string
        "response": "",
        "empathy_score": 0,
        "evaluation_feedback": "",
        "retry_count": 0
    }

    print(f"--- STARTING OPTIMIZATION (Max Retries: {max_retries}) ---")

    # Invoke the workflow
    final_state = optimizer_workflow.invoke(initial_state, {"recursion_limit": max_retries + 2})

    return final_state

In [93]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'which medication am I taking?'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)


In [94]:
summary_1 = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [95]:
summary_1

'The table shows medication records for a single patient (ID: 0fca905f-391c-08d3-4b93-b53f69b9da53) with five entries. Notably, all medications listed lack specific start and end dates, as well as refill information, indicating incomplete data. The medications include Acetaminophen, a combination of Acetaminophen and oxyCODONE, amLODIPine, and Naproxen sodium, with dosages provided for three entries. The table appears truncated, as it contains only five rows, and there may be additional relevant data not shown.'

In [96]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: which medication am I taking?
Final Score: 4/5
Final Response:
I understand that it can be a bit confusing to keep track of your medications, and I'm here to help you with that. Based on your records, you are currently taking the following medications:

1. **Acetaminophen 325 MG Oral Tablet (Tylenol)**
2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet**
3. **Amlodipine 2.5 MG Oral Tablet**
4. **Naproxen Sodium 220 MG Oral Tablet**

If you have any questions about these medications, such as their purposes or how to take them, please feel free to ask. It's important to feel comfortable and informed about your treatment.


**Test 2**




In [97]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'what has been my weight history'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [98]:
summary

'The data shows a trend of fluctuating body weight for the patient, with the most recent measurement on April 21, 2025, at 95.4 kg, indicating a decrease from 95.1 kg on April 15, 2024. The highest recorded weight was 102 kg in both April 2018 and April 2023. Notably, there are no entries for the years 2022 and 2023, suggesting a gap in data collection during that period. The table contains 11 rows of data.'

In [99]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: what has been my weight history
Final Score: 4/5
Final Response:
I understand that tracking your weight history can be important for your health and well-being, and I appreciate you reaching out about this. Here’s a summary of your weight history over the past several years:

- **2025-04-21**: 95.4 kg
- **2024-04-15**: 95.1 kg
- **2023-04-10**: 102 kg
- **2022-04-04**: 99.6 kg
- **2021-03-29**: 97.3 kg
- **2020-11-16**: 96.4 kg
- **2020-03-23**: 94.9 kg
- **2019-03-18**: 95.2 kg
- **2018-03-12**: 102 kg
- **2017-03-06**: 100 kg
- **2016-02-29**: 98.1 kg

It looks like your weight has fluctuated over the years, with some notable changes. If you have any specific concerns or goals regarding your weight, or if you would like to discuss this further, please let me know. I'm here to 

**Test 3**

In [100]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'List all my vital signs'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [101]:
summary

'The data for patient 0fca905f-391c-08d3-4b93-b53f69b9da53 includes 56 rows of vital measurements, with the most recent entries dated April 21, 2025. Notable trends include a decrease in Body Weight from 102 kg in April 2023 to 95.4 kg in April 2025, and a corresponding decrease in Body Mass Index (BMI) from 30.33 kg/m² to 28.35 kg/m² over the same period. Heart rate has fluctuated, with a notable drop from 88 beats per minute in April 2023 to 69 in April 2025. There are gaps in the data for Body Temperature, which is only recorded once in June 2017.'

In [102]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: List all my vital signs
Final Score: 4/5
Final Response:
I understand that you’re looking for a comprehensive overview of your vital signs, and I’m here to help you with that. It’s important to keep track of these measurements as they can provide valuable insights into your health.

Here’s a summary of your vital signs:

1. **Body Height**: 
   - 183.4 cm (most recent measurement on April 21, 2025)

2. **Body Weight**: 
   - 95.4 kg (most recent measurement on April 21, 2025)

3. **Body Mass Index (BMI)**: 
   - 28.35 kg/m² (most recent measurement on April 21, 2025)

4. **Heart Rate**: 
   - 69 beats per minute (most recent measurement on April 21, 2025)

5. **Respiratory Rate**: 
   - 14 breaths per minute (most recent measurement on April 21, 2025)

If you have any specific c

**Test 4**

In [103]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'Show my BMI trend'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [104]:
summary

'The data shows a trend of decreasing Body Mass Index (BMI) values over the last several years, with the most recent measurement on April 21, 2025, at 28.35 kg/m², down from a peak of 30.33 kg/m² on April 10, 2023. Notably, the BMI has decreased from 30.32 kg/m² in March 2018 to the current value, indicating a positive trend in weight management. There are no missing values in the shown data, but the table is truncated, displaying only 11 rows.'

In [105]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: Show my BMI trend
Final Score: 4/5
Final Response:
I understand that tracking your BMI trend can be important for your health and wellness journey, and I appreciate you reaching out for this information. Here’s a summary of your BMI values over the past few years:

- **2025-04-21**: 28.35 kg/m²
- **2024-04-15**: 28.26 kg/m²
- **2023-04-10**: 30.33 kg/m²
- **2022-04-04**: 29.62 kg/m²
- **2021-03-29**: 28.91 kg/m²
- **2020-11-16**: 28.66 kg/m²
- **2020-03-23**: 28.21 kg/m²
- **2019-03-18**: 28.31 kg/m²
- **2018-03-12**: 30.32 kg/m²
- **2017-03-06**: 29.74 kg/m²
- **2016-02-29**: 29.16 kg/m²

From this data, we can see that your BMI has shown some fluctuations over the years, with a recent decrease from 30.33 kg/m² in 2023 to 28.35 kg/m² in 2025. It’s great to see this positive tre

**Test 5**

In [106]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'Give me my blood pressure readings over time'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 No data found for the requested query.


In [107]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: Give me my blood pressure readings over time
Final Score: 4/5
Final Response:
I understand that keeping track of your blood pressure readings is important for your health, and it can be concerning if you feel unsure about your numbers. Unfortunately, I don't have access to your specific blood pressure readings over time. 

However, I recommend checking with your healthcare provider or the system where your readings are recorded. They should be able to provide you with a detailed history of your blood pressure measurements. If you have been keeping a log or using a home monitor, reviewing that information can also be very helpful.

If you have any other questions or need assistance with understanding your readings, please feel free to ask. Your health is important, and I'm here t

**Test 6**

In [108]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'What conditions have I been diagnosed with?'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 The table shows 21 rows of patient data, with a mix of resolved and active conditions. Notably, there are several active findings related to employment status, social factors, and chronic conditions, with the most recent entries dated April 2025. Resolved conditions include various dental and viral disorders, indicating a history of acute health issues. There are gaps in the resolution dates for several active findings, particularly concerning chronic pain and obesity, which may require further attention.


In [109]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: What conditions have I been diagnosed with?
Final Score: 4/5
Final Response:
I understand that it can be overwhelming to keep track of your health conditions, and I'm here to help you with that. Based on your records, here are the conditions you have been diagnosed with:

**Active Conditions:**
1. **Body Mass Index 30+ - Obesity**
2. **Chronic Low Back Pain**
3. **Essential Hypertension**
4. **Social Isolation**
5. **Limited Social Contact**
6. **Victim of Intimate Partner Abuse**
7. **Stress**
8. **Chronic Pain**
9. **Has a Criminal Record**

**Resolved Conditions:**
1. **Primary Dental Caries**
2. **Gingival Disease**
3. **Acute Viral Pharyngitis**
4. **Viral Sinusitis**
5. **Gingivitis**
6. **Traumatic Dislocation of Temporomandibular Joint**
7. **Medication Review Due**

If 

**Test 7**

In [110]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'How screwed up am I? '

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 The table shows 21 rows of patient data, with a mix of resolved and active conditions. Notably, there are several active findings related to employment status, social factors, and chronic conditions, with the most recent entries dated April 2025. Resolved conditions include various dental and viral disorders, indicating a history of acute health issues. There are gaps in the resolution dates for several active findings, including body mass index, social isolation, and chronic pain, suggesting ongoing health concerns.


In [111]:
df

,condition_name,first_noted_at,resolved_at,status
0,Primary dental caries (disorder),2020-11-30 16:55:05+00:00,2020-11-30 17:31:39+00:00,resolved
1,Full-time employment (finding),2025-04-21 14:58:06+00:00,NaT,active
2,Part-time employment (finding),2024-04-15 14:52:00+00:00,2025-04-21 14:58:06+00:00,resolved
3,Body mass index 30+ - obesity (finding),2009-07-13 14:19:34+00:00,NaT,active
4,Gingival disease (disorder),2021-04-12 16:42:45+00:00,2021-04-12 18:22:50+00:00,resolved
5,Acute viral pharyngitis (disorder),2017-06-25 13:19:34+00:00,2017-07-03 05:19:34+00:00,resolved
6,Has a criminal record (finding),2024-04-15 14:52:00+00:00,NaT,active
7,Chronic low back pain (finding),2014-02-05 14:19:34+00:00,NaT,active
8,Medication review due (situation),2025-04-21 14:19:34+00:00,2025-04-21 14:19:34+00:00,resolved
9,Social isolation (finding),2015-02-23 14:57:50+00:00,NaT,active


In [112]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 5/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: How screwed up am I? 
Final Score: 5/5
Final Response:
I want to start by acknowledging how you're feeling right now. It’s completely understandable to have concerns about your health and well-being, especially when facing multiple challenges. It can feel overwhelming at times, but please know that you are not alone in this.

Looking at your health snapshot, it seems you have had some resolved issues in the past, which is a positive sign. However, there are also several active conditions that may be contributing to your feelings of being "screwed up." It’s important to remember that having health challenges does not define you as a person, and there are steps you can take to improve your situation.

Here are some gentle next steps you might consider:

1. **Reach Out for Support*

**Test 8**

In [113]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'LIST my active prescriptions'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 The table shows medication data for a single patient (ID: 0fca905f-391c-08d3-4b93-b53f69b9da53) with five entries. All medications listed, including Acetaminophen and amLODIPine, have unspecified start and end dates, and no refills are recorded. Notably, the dose for Naproxen sodium is not provided, and one entry is completely blank. The data appears truncated, as only five rows are shown.


In [114]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: LIST my active prescriptions
Final Score: 4/5
Final Response:
I understand that keeping track of your medications can be overwhelming at times, and I'm here to help you with that. Here’s a list of your active prescriptions:

1. **Acetaminophen 325 MG Oral Tablet (Tylenol)** - 1 tablet
2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet** - 1 tablet
3. **Amlodipine 2.5 MG Oral Tablet** - 1 tablet
4. **Naproxen Sodium 220 MG Oral Tablet**

If you have any questions about these medications or need assistance with anything else, please feel free to ask. Your health and comfort are very important to us.


**Test 9**

In [115]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'Which of my visits have lab results'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 The data for patient ID 0fca905f-391c-08d3-4b93-b53f69b9da53 includes 15 encounters, all classified as ambulatory (AMB). Notably, there are gaps in the "reason" field for all encounters, indicating a lack of documented reasons for these visits. The most recent encounter occurred on April 15, 2024, while the earliest was on March 12, 2018. Additionally, there is a future encounter scheduled for April 21, 2025. The table appears to be truncated, as it may contain more rows beyond the 15 shown.


In [116]:
df

,patient_id,start_datetime,end_datetime,encounter_class,reason
0,0fca905f-391c-08d3-4b93-b53f69b9da53,2024-04-15 14:19:34+00:00,2024-04-15 14:52:00+00:00,AMB,None
1,0fca905f-391c-08d3-4b93-b53f69b9da53,2022-05-04 14:19:34+00:00,2022-05-04 14:34:34+00:00,AMB,None
2,0fca905f-391c-08d3-4b93-b53f69b9da53,2023-04-10 14:19:34+00:00,2023-04-10 15:04:29+00:00,AMB,None
3,0fca905f-391c-08d3-4b93-b53f69b9da53,2022-04-04 14:19:34+00:00,2022-04-04 15:10:09+00:00,AMB,None
4,0fca905f-391c-08d3-4b93-b53f69b9da53,2018-03-12 14:19:34+00:00,2018-03-12 14:58:41+00:00,AMB,None
5,0fca905f-391c-08d3-4b93-b53f69b9da53,2020-03-23 14:19:34+00:00,2020-03-23 15:12:46+00:00,AMB,None
6,0fca905f-391c-08d3-4b93-b53f69b9da53,2016-02-29 14:19:34+00:00,2016-02-29 14:58:08+00:00,AMB,None
7,0fca905f-391c-08d3-4b93-b53f69b9da53,2022-09-01 14:19:34+00:00,2022-09-01 14:34:34+00:00,AMB,None
8,0fca905f-391c-08d3-4b93-b53f69b9da53,2021-03-29 14:19:34+00:00,2021-03-29 14:53:45+00:00,AMB,None
9,0fca905f-391c-08d3-4b93-b53f69b9da53,2025-04-21 14:19:34+00:00,2025-04-21 14:58:06+00:00,AMB,None


In [117]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: Which of my visits have lab results
Final Score: 4/5
Final Response:
I understand that you're looking for information about your visits that have lab results, and I can see how important it is to have that information readily available. 

Unfortunately, I don't have direct access to the specific lab results associated with your visits. However, I recommend checking with your healthcare provider or the medical facility where you had your appointments. They should be able to provide you with a detailed list of your visits that include lab results.

If you have any other questions or need further assistance, please feel free to ask. Your health and peace of mind are important!
